# Gemma 2 - 9b
We train a simple classifier in this using computed embeddings as input from [here](https://www.kaggle.com/code/kishanvavdara/gemma-2-9b-part-1?scriptVersionId=186083288) and compute embeddings for test and use trained classifier for inference. Let's get started!

Upvote if you found this helpful!

# Import Libs

In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip
!pip install -q -U transformers --no-index --find-links ../input/libs-install

In [ ]:
import os
import gc
import re
from time import time

import torch
import transformers
import sklearn
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from transformers import Gemma2ForCausalLM, GemmaTokenizer, BitsAndBytesConfig

import time
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

from torch.cuda.amp import autocast
from threading import Thread

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

if (not torch.cuda.is_available()): print("Sorry - GPU required!")

# Train Classifier 

In [ ]:
train_df = pd.read_csv('/kaggle/input/gemma-2-9b-part-1/train_embed.csv')
train_embed = np.load('/kaggle/input/gemma-2-9b-part-1/gemma2_train_embed.npy')

train_df.loc[:, 'label'] = np.argmax(train_df[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)

In [ ]:
# splits
Targets = ['winner_model_a','winner_model_b','winner_tie']

y = train_df['label'].values
train_idx, test_idx = train_test_split(train_df.index, test_size=0.1, random_state=42, stratify=y)

X_train, y_train = train_embed[train_idx], train_df.iloc[train_idx]['label'].values
X_test, y_test = train_embed[test_idx], train_df.iloc[test_idx]['label'].values

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# Here we just use classifier with default settings, try out tuning the params) 

model_cb = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    early_stopping_rounds=10,
    task_type='GPU',
    devices='0:1',
    verbose=100)

model_cb.fit(X_train, y_train, 
          eval_set=(X_test, y_test),
          early_stopping_rounds=50)

In [ ]:
y_pred_proba = model_cb.predict_proba(X_test)
y_pred = model_cb.predict(X_test)

# Evaluate the model
logloss = log_loss(y_test, y_pred_proba)
accuracy = accuracy_score(y_test, y_pred)
gc.collect()

print(f'Log Loss: {logloss:.3f}')
print(f'Accuracy: {accuracy:.3f}')

We'll use this classifer for inference.

# Load Gemma 2

In [ ]:
MODEL_PATH = '/kaggle/input/gemma-2-9b-hf'
MAX_LENGTH = 1024
BATCH_SIZE = 2
    
device0 = torch.device('cuda:0')
device1 = torch.device('cuda:1')

tokenizer = GemmaTokenizer.from_pretrained(MODEL_PATH)

bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False)

model_0 = Gemma2ForCausalLM.from_pretrained(MODEL_PATH,
                                        revision="float16",
                                        device_map='cuda:0',
                                        quantization_config=bnb_config_4bit)        

model_1 = Gemma2ForCausalLM.from_pretrained(MODEL_PATH,
                                        revision="float16",
                                        device_map='cuda:1',
                                        quantization_config=bnb_config_4bit)     

In [ ]:
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return sentences[-1] if sentences else ''
  
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

test['text'] = '<start_of_turn>User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n----\n\nModel B:\n'  + test['response_b'] + '<end_of_turn><eos>'
print(test['text'][0])

# Tokenize 

In [ ]:
tokens = tokenizer(test['text'].tolist(),
                   padding='max_length',
                   max_length=MAX_LENGTH,
                   truncation=True,
                   return_tensors='pt')


data = pd.DataFrame()
data['INPUT_IDS'] = [tensor.tolist() for tensor in tokens['input_ids']]
data['ATTENTION_MASKS'] = [tensor.tolist() for tensor in  tokens['attention_mask']]
data[:2]

# Get embeddings

In [ ]:
def get_embeddings(df, model, device, batch_size=BATCH_SIZE):  
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)

    embed_list = []

    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch_input_ids = input_ids[start_idx:end_idx].to(device)
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)
        gc.collect()
        torch.cuda.empty_cache()
        with torch.no_grad():
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, output_hidden_states=True)
            embed = outputs.hidden_states[-1]
            embed_mean = torch.mean(embed, dim=1).cpu() #mean pool
            embed_list.append(embed_mean) 
            
            torch.cuda.empty_cache()
        
    embeddings = torch.cat(embed_list, dim=0)
    return embeddings

def compute_embed(df, model, device, results, index):
    results[index] = get_embeddings(df, model, device)

In [ ]:
st = time.time()

N_SAMPLES = len(data)
half = round(N_SAMPLES / 2)
sub1 = data.iloc[0:half].copy()
sub2 = data.iloc[half:N_SAMPLES].copy()

results = {}

t0 = Thread(target=compute_embed, args=(sub1, model_0, device0, results, 0))
t1 = Thread(target=compute_embed, args=(sub2, model_1, device1, results, 1))

t0.start()
t1.start()

t0.join()
t1.join()

print(f"Processing complete. Total time: {time.time() - st:.2f} seconds")

In [ ]:
test_embeddings = torch.cat([results[0], results[1]], dim=0)
test_embeddings.shape

In [ ]:
gc.collect()
del model_1
del  model_0
torch.cuda.empty_cache()

# Inference

In [ ]:
preds = model_cb.predict_proba(test_embeddings.numpy())
preds

# Submission

In [ ]:
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
sample_sub[Targets] =  preds

display(sample_sub)

In [ ]:
sample_sub.to_csv('submission.csv', index=False)

# Conclusion 

That's it! Just wanted to share the idea! Try out tuning the classifier or using other classifier. Thanks!

If you learned something, Please upvote:)